In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import xarray as xr
import plotly.graph_objects as go
import plotly.express as px
from shapely.geometry import Point
from scipy.interpolate import griddata


In [2]:

# Load data and create a GeoDataFrame
ec_data = pd.read_excel("../Data/ec_data/2024_TAPS_Veris_raw_spatial_data.xlsx")
geometry = [Point(xy) for xy in zip(ec_data['Long'], ec_data['Lat'])]
ec_vector_data = gpd.GeoDataFrame(ec_data, geometry=geometry)
ec_vector_data.set_crs("EPSG:4326", inplace=True) 

# Interpolate data
points = np.array(list(zip(ec_vector_data.geometry.x, ec_vector_data.geometry.y)))
ec_shallow = ec_vector_data['EC SH'].values
ec_deep = ec_vector_data['EC DP'].values

# Define grid parameters for interpolation
x_min, x_max = points[:, 0].min(), points[:, 0].max()
y_min, y_max = points[:, 1].min(), points[:, 1].max()
grid_x, grid_y = np.mgrid[x_min:x_max:200j, y_min:y_max:200j]

# Interpolated grids
grid_z_shallow = griddata(points, ec_shallow, (grid_x, grid_y), method='cubic')
grid_z_deep = griddata(points, ec_deep, (grid_x, grid_y), method='cubic')

# Load plot boundary shapefile
plot_boundary = gpd.read_file("../Data/plot_boundaries/Map with all plots/2024_Colby_TAPS_Harvest_Area.shx")


In [4]:

# Plot with Plotly
fig = go.Figure()

# Add EC Shallow interpolated data as a contour plot
fig.add_trace(go.Contour(
    z=grid_z_shallow,
    x=np.linspace(x_min, x_max, grid_x.shape[0]), # Longitude range
    y=np.linspace(y_min, y_max, grid_y.shape[1]), # Latitude range
    colorscale="Inferno",
    colorbar=dict(title="EC Shallow"),
    contours=dict(showlines=False)
))

# Add plot boundaries
# Add plot boundaries with proper format
for _, row in plot_boundary.iterrows():
    boundary_x, boundary_y = list(row['geometry'].exterior.xy[0]), list(row['geometry'].exterior.xy[1])
    fig.add_trace(go.Scatter(
        x=boundary_x,
        y=boundary_y,
        mode="lines",
        line=dict(color="black", width=2),
        name="Plot Boundary"
    ))

# Add EC Deep interpolated data as a separate layer if desired
fig.add_trace(go.Contour(
    z=grid_z_deep,
    x=np.linspace(x_min, x_max, grid_x.shape[0]), 
    y=np.linspace(y_min, y_max, grid_y.shape[1]),
    colorscale="Blues",
    colorbar=dict(title="EC Deep"),
    contours=dict(showlines=False),
    opacity=0.5  # Adjust to layer the plots
))

# Update layout for clarity
fig.update_layout(
    title="Interpolated EC Data - Shallow and Deep",
    xaxis_title="Longitude",
    yaxis_title="Latitude",
    template="plotly_white"
)

# Display plot
fig.show()


In [ ]:
# import dash
# from dash import dcc, html, Input, Output
# import geopandas as gpd
# import plotly.graph_objects as go
# from scipy.interpolate import griddata
# import numpy as np
# import pandas as pd
# from shapely.geometry import Point

# # Load your data
# ec_data = pd.read_excel("../Data/ec_data/2024_TAPS_Veris_raw_spatial_data.xlsx")
# geometry = [Point(xy) for xy in zip(ec_data['Long'], ec_data['Lat'])]
# ec_vector_data = gpd.GeoDataFrame(ec_data, geometry=geometry)
# ec_vector_data.set_crs("EPSG:4326", inplace=True)

# plot_boundary = gpd.read_file("../Data/plot_boundaries/Map with all plots/2024_Colby_TAPS_Harvest_Area.shx")

# # Interpolation for EC Shallow and Deep
# points = np.array(list(zip(ec_vector_data.geometry.x, ec_vector_data.geometry.y)))
# ec_shallow = ec_vector_data['EC SH'].values
# ec_deep = ec_vector_data['EC DP'].values

# # Define grid parameters for interpolation
# x_min, x_max = points[:, 0].min(), points[:, 0].max()
# y_min, y_max = points[:, 1].min(), points[:, 1].max()
# grid_x, grid_y = np.mgrid[x_min:x_max:200j, y_min:y_max:200j]

# grid_z_shallow = griddata(points, ec_shallow, (grid_x, grid_y), method='cubic')
# grid_z_deep = griddata(points, ec_deep, (grid_x, grid_y), method='cubic')

# # Dash App
# app = dash.Dash(__name__)

# app.layout = html.Div([
#     dcc.Dropdown(
#         id='plot-selector',
#         options=[{'label': f"Plot {i}", 'value': i} for i in plot_boundary.index],
#         value=plot_boundary.index[0],
#         placeholder="Select a plot"
#     ),
#     dcc.Graph(id='plot-graph')
# ])

# @app.callback(
#     Output('plot-graph', 'figure'),
#     Input('plot-selector', 'value')
# )
# def update_plot(selected_plot):
#     fig = go.Figure()

#     # Extract the selected plot boundary
#     boundary = plot_boundary.loc[selected_plot, 'geometry']
#     boundary_x, boundary_y = list(boundary.exterior.xy[0]), list(boundary.exterior.xy[1])

#     # Add boundary to the map
#     fig.add_trace(go.Scatter(
#         x=boundary_x,
#         y=boundary_y,
#         mode="lines",
#         line=dict(color="black", width=2),
#         name=f"Plot {selected_plot} Boundary"
#     ))

#     # Add EC Shallow and Deep as contour plots
#     fig.add_trace(go.Contour(
#         z=grid_z_shallow,
#         x=np.linspace(x_min, x_max, grid_x.shape[0]),
#         y=np.linspace(y_min, y_max, grid_y.shape[1]),
#         colorscale="Inferno",
#         colorbar=dict(title="EC Shallow"),
#         contours=dict(showlines=False)
#     ))

#     fig.add_trace(go.Contour(
#         z=grid_z_deep,
#         x=np.linspace(x_min, x_max, grid_x.shape[0]),
#         y=np.linspace(y_min, y_max, grid_y.shape[1]),
#         colorscale="Blues",
#         colorbar=dict(title="EC Deep"),
#         contours=dict(showlines=False),
#         opacity=0.5
#     ))

#     # Update layout
#     fig.update_layout(
#         title=f"EC Data for Plot {selected_plot}",
#         xaxis_title="Longitude",
#         yaxis_title="Latitude",
#         template="plotly_white"
#     )

#     return fig

# if __name__ == '__main__':
#     app.run_server(debug=True)


# Mahek below is the code you are going to use

In [3]:
import dash
from dash import dcc, html, Input, Output
import geopandas as gpd
import plotly.graph_objects as go
import rioxarray  # For raster clipping
from scipy.interpolate import griddata
import numpy as np
import xarray as xr
import pandas as pd
from shapely.geometry import Point

# Load EC data and create GeoDataFrame
ec_data = pd.read_excel("../Data/ec_data/2024_TAPS_Veris_raw_spatial_data.xlsx")
geometry = [Point(xy) for xy in zip(ec_data['Long'], ec_data['Lat'])]
ec_vector_data = gpd.GeoDataFrame(ec_data, geometry=geometry)
ec_vector_data.set_crs("EPSG:4326", inplace=True)

# Load plot boundary shapefile
plot_boundary = gpd.read_file("../Data/plot_boundaries/Map with all plots/2024_Colby_TAPS_Harvest_Area.shx")

# Interpolate EC Shallow and Deep
points = np.array(list(zip(ec_vector_data.geometry.x, ec_vector_data.geometry.y)))
ec_shallow = ec_vector_data['EC SH'].values
ec_deep = ec_vector_data['EC DP'].values

# Define grid parameters for interpolation
x_min, x_max = points[:, 0].min(), points[:, 0].max()
y_min, y_max = points[:, 1].min(), points[:, 1].max()
grid_x, grid_y = np.mgrid[x_min:x_max:200j, y_min:y_max:200j]

# Interpolated grids
grid_z_shallow = griddata(points, ec_shallow, (grid_x, grid_y), method='cubic')
grid_z_deep = griddata(points, ec_deep, (grid_x, grid_y), method='cubic')

# Convert interpolated grids to xarray DataArrays for clipping
ec_shallow_da = xr.DataArray(grid_z_shallow, dims=("y", "x"), 
                             coords={"y": np.linspace(y_min, y_max, grid_y.shape[1]), 
                                     "x": np.linspace(x_min, x_max, grid_x.shape[0])})
ec_deep_da = xr.DataArray(grid_z_deep, dims=("y", "x"), 
                          coords={"y": np.linspace(y_min, y_max, grid_y.shape[1]), 
                                  "x": np.linspace(x_min, x_max, grid_x.shape[0])})

# Set CRS to match the plot boundary CRS
ec_shallow_da.rio.set_crs("EPSG:4326")
ec_deep_da.rio.set_crs("EPSG:4326")
print('Inferno_r')



plot_boundary.head()

Inferno_r


,Name,Block_ID,TRT_ID,Plot_ID,geometry
0,Span D,1,15,2502,"POLYGON ((-101.06572 39.387, -101.06572 39.386..."
1,Span A,4,27,204,"POLYGON ((-101.06495 39.38511, -101.06495 39.3..."
2,Span A,4,30,206,"POLYGON ((-101.06417 39.38511, -101.06417 39.3..."
3,Span A,4,2,205,"POLYGON ((-101.06456 39.38511, -101.06456 39.3..."
4,Span A,2,14,203,"POLYGON ((-101.06534 39.38512, -101.06534 39.3..."


In [ ]:

fig = go.Figure()
selected_plot = plot_boundary.index[0]
# Extract selected plot boundary
selected_boundary = plot_boundary.loc[[selected_plot], 'geometry']

# clip_fn = lambda polygon, R: R.rio.clip( [polygon.geometry], 
#                                     crs = R.rio.crs,
#                                     all_touched = True)

# Clip EC shallow and deep rasters to the selected plot boundary
ec_shallow_clipped = ec_shallow_da.rio.clip(selected_boundary.geometry, drop=True,all_touched = True)
ec_deep_clipped = ec_deep_da.rio.clip(selected_boundary.geometry, drop=True,all_touched = True)

# Plot EC Shallow
fig.add_trace(go.Heatmap(
    z=ec_shallow_clipped.values,
    x=ec_shallow_clipped.coords['x'].values,
    y=ec_shallow_clipped.coords['y'].values,
    colorscale="Inferno_r",
    colorbar=dict(title="EC Shallow"),
    zsmooth="best"
))


# Add plot boundary outline
boundary_x, boundary_y = list(selected_boundary.geometry.iloc[0].exterior.xy[0]), list(selected_boundary.geometry.iloc[0].exterior.xy[1])
fig.add_trace(go.Scatter(
    x=boundary_x,
    y=boundary_y,
    mode="lines",
    line=dict(color="black", width=2),
    name="Plot Boundary"
))

# Update layout
fig.update_layout(
    title=f"Clipped EC Data for Plot {selected_plot}",
    xaxis_title="Longitude",
    yaxis_title="Latitude",
    template="plotly_white"
)


In [ ]:

fig = go.Figure()
selected_plot = plot_boundary.index[0]
# Extract selected plot boundary
selected_boundary = plot_boundary.loc[[selected_plot], 'geometry']

# clip_fn = lambda polygon, R: R.rio.clip( [polygon.geometry], 
#                                     crs = R.rio.crs,
#                                     all_touched = True)

# Clip EC shallow and deep rasters to the selected plot boundary
ec_shallow_clipped = ec_shallow_da.rio.clip(selected_boundary.geometry, drop=True,all_touched = True)
ec_deep_clipped = ec_deep_da.rio.clip(selected_boundary.geometry, drop=True,all_touched = True)

# Plot EC Shallow
fig.add_trace(go.Heatmap(
    z=ec_shallow_clipped.values,
    x=ec_shallow_clipped.coords['x'].values,
    y=ec_shallow_clipped.coords['y'].values,
    colorscale="Inferno_r",
    colorbar=dict(title="EC Shallow"),
    zsmooth="best"
))


# Add plot boundary outline
boundary_x, boundary_y = list(selected_boundary.geometry.iloc[0].exterior.xy[0]), list(selected_boundary.geometry.iloc[0].exterior.xy[1])
fig.add_trace(go.Scatter(
    x=boundary_x,
    y=boundary_y,
    mode="lines",
    line=dict(color="black", width=2),
    name="Plot Boundary"
))

# Update layout
fig.update_layout(
    title=f"Clipped EC Data for Plot {selected_plot}",
    xaxis_title="Longitude",
    yaxis_title="Latitude",
    template="plotly_white"
)


In [ ]:

# # Dash app setup
# app = dash.Dash(__name__)

# app.layout = html.Div([
#     dcc.Dropdown(
#         id='plot-selector',
#         options=[{'label': f"Plot {i}", 'value': i} for i in plot_boundary.index],
#         value=plot_boundary.index[0],
#         placeholder="Select a plot"
#     ),
#     dcc.Graph(id='plot-graph')
# ])

# @app.callback(
#     Output('plot-graph', 'figure'),
#     Input('plot-selector', 'value')
# )
# def update_plot(selected_plot):
#     fig = go.Figure()

#     # Extract selected plot boundary
#     selected_boundary = plot_boundary.loc[[selected_plot], 'geometry']

#     # clip_fn = lambda polygon, R: R.rio.clip( [polygon.geometry], 
#     #                                     crs = R.rio.crs,
#     #                                     all_touched = True)

#     # Clip EC shallow and deep rasters to the selected plot boundary
#     ec_shallow_clipped = ec_shallow_da.rio.clip(selected_boundary.geometry, drop=True,all_touched = True)
#     ec_deep_clipped = ec_deep_da.rio.clip(selected_boundary.geometry, drop=True,all_touched = True)

#     # Plot EC Shallow
#     fig.add_trace(go.Heatmap(
#         z=ec_shallow_clipped.values,
#         x=ec_shallow_clipped.coords['x'].values,
#         y=ec_shallow_clipped.coords['y'].values,
#         colorscale="Inferno_r",
#         colorbar=dict(title="EC Shallow"),
#         zsmooth="best"
#     ))

#     # Plot EC Deep
#     # fig.add_trace(go.Heatmap(
#     #     z=ec_deep_clipped.values,
#     #     x=ec_deep_clipped.coords['x'].values,
#     #     y=ec_deep_clipped.coords['y'].values,
#     #     colorscale="Inferno_r",
#     #     colorbar=dict(title="EC Deep"),
#     #     zsmooth="best",
#     #     opacity=0.5
#     # ))

#     # Add plot boundary outline
#     boundary_x, boundary_y = list(selected_boundary.geometry.iloc[0].exterior.xy[0]), list(selected_boundary.geometry.iloc[0].exterior.xy[1])
#     fig.add_trace(go.Scatter(
#         x=boundary_x,
#         y=boundary_y,
#         mode="lines",
#         line=dict(color="black", width=2),
#         name="Plot Boundary"
#     ))

#     # Update layout
#     fig.update_layout(
#         title=f"Clipped EC Data for Plot {selected_plot}",
#         xaxis_title="Longitude",
#         yaxis_title="Latitude",
#         template="plotly_white"
#     )

#     return fig

# if __name__ == '__main__':
#     app.run_server(debug=True)
